## Understanding the Datasets

### Train vs. Test
In this competition, you’ll gain access to two datasets that are samples of past specialization subscriptions that contain information about the learner, the specialization, and the learner's activity in the subscription thus far. One dataset is titled `train.csv` and the other is titled `test.csv`.

`train.csv` contains 60% of the overall sample and importantly, will reveal whether or not the subscription was continued into the next month (the “ground truth”). The test and validation sets contain 20% of the overall sample, respectively. 

Using the patterns one can find in the `train.csv` data, the model will predict whether the subscriptions in `test.csv` will be continued for another month, or not.

### Dataset descriptions
Both `train.csv` and `test.csv` contain one row for each unique specialization subscription. For each subscription, a single observation (`subscription_id`) is included as of a particular date (`observation_dt`) during which the subscription was active. This date was chosen at random from all the dates during which the subscription was active. In some instances it is soon after the subscription was initiated; in other instances, it is several months after the subscription was initiated and after several previous payments were made. Therefore, the model will have to be able to adapt to different stages of the subscription.

In addition to those identifier columns, the datasets also contain the target label for the task, a binary column `is_retained`.

In [2]:
import pandas as pd
data_descriptions = pd.read_csv('data_descriptions.csv')
pd.set_option('display.max_colwidth', None)
data_descriptions

,Column_name,Column_type,Data_type,Description
0,subscription_id,Identifier,character,Unique identifier of each subscription
1,observation_dt,Identifier,date,The date on which the subscription was observed to calculate the features in the dataset. It was chosen at random amongst all the dates between the start of the subscription and the end of the subscription (before cancellation)
2,is_retained,Target,Integer,"TRAINING SET ONLY! 0 = the learner cancelled their subscription before next payment, 1 = the learner made an additional payment in this subscription"
3,specialization_id,Feature - Specialization Info,character,Unique identifier of a specialization (each subscription gives a learner access to a particular specialization)
4,cnt_courses_in_specialization,Feature - Specialization Info,integer,number of courses in the specialization
5,specialization_domain,Feature - Specialization Info,character,"primary domain of the specialization (Computer Science, Data Science, etc.)"
6,is_professional_certificate,Feature - Specialization Info,boolean,"BOOLEAN for whether the specialization is a ""professional certicate"" (a special type of specialization that awards completers with an industry-sponsored credential)"
7,is_gateway_certificate,Feature - Specialization Info,boolean,"BOOLEAN for whether the specialization is a ""gateway certificate"" (a special type of specialization geared towards learners starting in a new field)"
8,learner_days_since_registration,Feature - Learner Info,integer,Days from coursera registration date to the date on which the observation is made
9,learner_country_group,Feature - Learner Info,character,"the region of the world that the learner is from (United States, East Asia, etc.)"


In [5]:
# Import required packages

# Data packages
import pandas as pd
import numpy as np
import tensorflow as tf

# Machine Learning / Classification packages
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

# Visualization Packages
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
# Import any other packages you may want to use
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.activations import linear, relu, sigmoid


## Load the Data

In [33]:
# Get 60% of the dataset as the training set. Put the remaining 40% in a temporary data frame: temp
# Split the 40% subset above into two: one half for cross validation and the other for the test set
# train, temp = train_test_split(train_df, test_size=0.4, random_state = 1234)
# validation, test = train_test_split(temp, test_size = 0.5, random_state = 1234)

# del temp

# train.to_csv("training.csv")
# validation.to_csv("validation.csv")
# test.to_csv("test.csv")

In [8]:
training_data = pd.read_csv("training.csv", low_memory = False)
validation_data = pd.read_csv("validation.csv")
test_data = pd.read_csv("test.csv")

### preprocessing the data sets

In [12]:

def pre_processing(path, frac_features, frac_features_names, drop_features):
#     drop_features: list of strings with the names of the categorical features 
#     binary_features: list of string witht the names of the binary columns
#     target_col: string

    df = pd.read_csv(path, low_memory = False)
    df = df.dropna()
    
    total_courses_count = "cnt_courses_in_specialization"
    
    for feature, name in zip(frac_features, frac_features_names):
        
        df[name] = df[feature] / df[total_courses_count]

    df = df.drop(columns = drop_features)
    
    return df
    
def scaling(df, binary_features, is_train): 
    
# scaling the data using the standard scaler
    binary_cols = df[binary_features].values
    binary_cols = binary_cols.astype(int)
    cols_to_scale = list(set(df.columns).difference(set(binary_features)))
    
    if is_train:
    
        cols_scaled = scaler.fit_transform(df[cols_to_scale])
        
    else:
        
        cols_scaled = scaler.transform(df[cols_to_scale])
        
    scaled_df = np.concatenate((binary_cols, cols_scaled), axis = 1)
    
    return scaled_df


    

In [13]:
# convert integers of course enrollment to fractions
frac_features = ["cnt_enrollments_started_before_payment_period", 
                                    "cnt_enrollments_completed_before_payment_period", 
                                    "cnt_enrollments_active_before_payment_period", 
                                    "cnt_enrollments_started_during_payment_period", 
                                    "cnt_enrollments_completed_during_payment_period", 
                                    "cnt_enrollments_active_during_payment_period",]

frac_features_names = ["frc_enrollments_started_before_payment_period", 
                       "frc_enrollments_completed_before_payment_period", 
                       "frc_enrollments_active_before_payment_period", 
                       "frc_enrollments_started_during_payment_period", 
                       "frc_enrollments_completed_during_payment_period", 
                       "frc_enrollments_active_during_payment_period"]

# drop the categorical features
drop_features = ["specialization_id", "specialization_domain", "learner_country_group", 
                                    "learner_gender", "observation_dt", "subscription_id", 
                                    "learner_cnt_other_transactions_past", 
                                    "cnt_enrollments_started_before_payment_period", 
                                    "cnt_enrollments_completed_before_payment_period", 
                                    "cnt_enrollments_active_before_payment_period", 
                                    "cnt_enrollments_started_during_payment_period", 
                                    "cnt_enrollments_completed_during_payment_period", 
                                    "cnt_enrollments_active_during_payment_period", 
                                    "cnt_courses_in_specialization",]

binary_features = ["is_professional_certificate", "is_gateway_certificate", 
                   "is_subscription_started_with_free_trial", "is_active_capstone_during_pay_period",]

target_col = ["is_retained"]

In [14]:
# preprocessing training set
path_train = "training.csv"
train_data = pre_processing(path_train, frac_features, frac_features_names, drop_features)

y_train = train_data["is_retained"].values
train_data = train_data.drop(columns = "is_retained")

# scaling training data
scaler = StandardScaler()
X_train = scaling(train_data, binary_features, 1 == 1)

In [15]:
# preprocessing validation set
path_val = "validation.csv"
val_data = pre_processing(path_val, frac_features, frac_features_names, drop_features)

y_val = val_data["is_retained"].values
val_data = val_data.drop(columns = "is_retained")

# scaling validation data
X_val = scaling(val_data, binary_features, 1 == 0)

In [16]:
# preprocessing test set
path_test = "test.csv"
test_df = pd.read_csv(path_test)
test_ids = test_df[['subscription_id']]
test_data = pre_processing(path_test, frac_features, frac_features_names, drop_features)

y_test = test_data["is_retained"].values
test_data = test_data.drop(columns = "is_retained")

# scaling test data
X_test = scaling(test_data, binary_features, 1 == 0)



### model

In [21]:
model = Sequential(
    [
        Dense(256, input_shape = (X_train.shape[1],), activation = "relu"),
        Dense(128, activation = "relu"),
        Dense(64, activation = "relu"),
        Dense(1, activation = "sigmoid")
    ]
)


In [22]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(0.001),
    metrics = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)

model.fit(
    X_train,y_train,
    validation_data = (X_val, y_val),
    epochs = 100,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        mode = "min",
        patience = 2,
        verbose = 0,
        restore_best_weights = True,
    ),
                ]
)

Epoch 1/100
7762/7762 [==============================] - 17s 2ms/step - loss: 0.5963 - binary_accuracy: 0.6775 - precision: 0.6894 - recall: 0.7424 - val_loss: 0.5990 - val_binary_accuracy: 0.6740 - val_precision: 0.6980 - val_recall: 0.7018
Epoch 2/100
7762/7762 [==============================] - 16s 2ms/step - loss: 0.5897 - binary_accuracy: 0.6833 - precision: 0.6967 - recall: 0.7415 - val_loss: 0.5905 - val_binary_accuracy: 0.6819 - val_precision: 0.6876 - val_recall: 0.7564
Epoch 3/100
7762/7762 [==============================] - 17s 2ms/step - loss: 0.5877 - binary_accuracy: 0.6847 - precision: 0.6968 - recall: 0.7459 - val_loss: 0.5887 - val_binary_accuracy: 0.6855 - val_precision: 0.6952 - val_recall: 0.7467
Epoch 4/100
7762/7762 [==============================] - 18s 2ms/step - loss: 0.5862 - binary_accuracy: 0.6857 - precision: 0.6964 - recall: 0.7502 - val_loss: 0.5878 - val_binary_accuracy: 0.6855 - val_precision: 0.6910 - val_recall: 0.7582
Epoch 5/100
7762/7762 [=========

## Predictions

In [23]:
predictions = model.predict(X_test).flatten()

2588/2588 [==============================] - 3s 991us/step


In [24]:
# Combine predictions with label column into a dataframe

prediction_df = pd.DataFrame({'subscription_id': test_ids.values[:, 0], 
                             'predicted_probability': predictions})

In [25]:
prediction_df.head(10)

,subscription_id,predicted_probability
0,cmruPNNlEeu5xhIWn-rHGQ,0.437496
1,sxQIF1Q_EeyCJRL_5xXcrw,0.678398
2,8dAgJoKnEeuyFgrvXo0Zqw,0.787826
3,_i3b2spOEeuktA5pX5p5sQ,0.541132
4,UmAgAiquEeyEkBIiq6qFSQ,0.778768
5,QidbzStWEeyFfhIE_bgb3w,0.495438
6,fUT_lYj5EeuT6Q4G3AaHkQ,0.364273
7,GlZ8D5VbEeuQdQonodlz7Q,0.792161
8,NcRNmzzkEey-bw4pL2HaOQ,0.535541
9,zc1hoba_EeufeAq6C_fAOw,0.524613


In [26]:
# roc_auc_score

roc_auc_score(y_test, predictions)

0.7532098423804503